## 1. Importing Modules

The programs below will import all of the necessary modules used in the data pre-processing program.  Various functions and libraries will be called throughout the duration of the program.

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
#!pip install twython
from twython import Twython
from twython.exceptions import TwythonError, TwythonRateLimitError
import io, json, time, os, logging, argparse, atexit, gzip, sys, csv

In [19]:
#insert normal libraries
import pandas as pd
import re
import json
import numpy as np
import ast
import datetime as dt
import os.path

#insert webscraping tools
import requests
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from numpy import int64

#insert NLTK libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import *
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Matt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 2. Changing Folder Paths

The data-preprocessing program has been constructed in such a way where it constantly reads and exports data to your local folder paths.  It is recommended that Google Drive Backup & Sync (GDBS) is installed on your device if you are collaborating with multiple people while using this framework.  GDBS allows for your folders on Google Drive to be accessed locally by the program.

Please change you folder path names here.  Each folder path can be described in detail below:

**project_folder:** root folder name for all relevant files.<br>
**json_path:** folder to hold all exported JSON files from Twython.<br>
**converted_path:** deposit initial Excel files.<br>
**converted_url_path:** deposit final Excel files.<br>

In [5]:
project_folder = r'C:\Users\Matt\Google Drive\Think Luna' # <-- change root folder name

json_path = project_folder + r'\Data\02_Json' # <-- change folder location for your stored JSON files
converted_path = project_folder + r'\Data\converted_to_excel' # <-- change folder location for depositing your Excelfiles

converted_url_path = project_folder + r'\Data\converted_to_url' # <-- change folder location for depositing your Excelfiles

## 3. Keywords Filter List

Please enter a list of negative and positive keywords here.  These will be used to identify potential negative and positive vaccine sentiments in the database when you are ready to label the training data.  A "keyword_filter" column will appear at the end of the database which may speed up the labelling process and help to determine class targets.

A list of both sets of keywords has been identified below.  Please add any additional words that may be relevant.

In [6]:
neg_keywords = ["hearus","FDA", "poisoning", "ugly", "truth", "aborted", "tissue", "fetal", "population", "VAERS",
"africa", "fetal", "cells", "population", "control", "vaxxed", "fetus", "profit", "vaxxed", "force", "SIDS", 
"victim", "agenda", "fraud", "sterilization", "victims", "allergy", "free", "choice", "theft", "wake up", "aluminium", 
"freedom", "thimerosal", "wakefield", "aluminum", "Gates", "tissue", "warfare", "Bill", "gilead", "toxic", "weapon",
"black", "greed", "kill", "whistleblower", "chip", "hidden", "liars", "witnessed", "choice", "hoax", "mandate", 
"compensation", "injure", "manmade", "control", "injured", "man-made", "corrupt", "injuries", "manufacture", "damage", 
"injury", "merck", "damage", "insert", "mercury", "diabetes", "patent", "monetize", "disclose", "Poison", "natural", "engineer", 
"Patent", "truth"]

pos_keywords = ["flufighters", "flujab", "vaccineswork", "clinic", "public health", 
                "idweek19", "flu shot", "flushot", "fluvaccine", "flu jab", "flu vaccine", 
                "get your flu shot", "fluchampions", "flu war", "flu season"]

## 4. Data Extraction

This program uses Twitter data structured as a JSON filetype.  The problem with this however is that the data is not structured in the normal JSON format.  There are subsets of dictionaries embedded under certain headings (ie. the "User" heading contains multiple pieces of information such as "user_id", "friend_count" and "follower_count" all contained in one cell for one particular tweet).  This section of the program extracts the subsets of data from "user", "coordinates", and "place" headings, and appends it to the primary dataframe. 

#### Parsing Subsets of Data

In [7]:
#parses the 'user' heading into its own dataframe
def parse_user_data(df):
    
    #index user column and create blank dataframe
    idx = df.columns.get_loc("user")
    sub_df = pd.DataFrame()

    #individually parse out dictionary from user column
    for i in range(0,len(df)):
        sub_df = sub_df.append(pd.DataFrame(pd.DataFrame(df.iloc[i,idx]).iloc[0,:]).T)

    #reset indices in dataframe and list useful columns to keep
    sub_df.reset_index(drop=True, inplace=True)
    column_list = ['id','name','screen_name','location','description','url','protected','followers_count','friends_count',
                   'listed_count','created_at','favourites_count', 'utc_offset','time_zone', 'geo_enabled', 'verified', 
                   'statuses_count', 'lang','contributors_enabled']
    
    #build dataframe containing only useful columns
    sub_df = sub_df[column_list]
    sub_df.reset_index(drop=True, inplace=True)
    sub_df.columns = ['user_' + str(col) for col in sub_df.columns] #append "user" in front of all column names
    
    return sub_df

In [8]:
#parses the 'coordinates' heading into its own dataframe
def parse_coord_data(df):
    idx = df.columns.get_loc("coordinates")
    
    #replace none types with empty dictionaries
    thisdict = str({'type': "Point",'coordinates': ["NA","NA"]})
    df['coordinates'].replace([None], thisdict, inplace=True)

    sub_df_long = pd.DataFrame()
    sub_df_lat = pd.DataFrame()

    for i in range(0,len(df)):
        try:
            sub_df_long = sub_df_long.append(pd.DataFrame(df.iloc[i,idx]).iloc[0,:])
        except:
            sub_df_long = sub_df_long.append(pd.DataFrame(ast.literal_eval(df.iloc[i,idx])).iloc[0,:])
            
    
    sub_df_long['long'] = sub_df_long['coordinates']
    sub_df_long.drop(['coordinates','type'],axis = 1,inplace = True)
    sub_df_long.reset_index(drop=True, inplace=True)

    for i in range(0,len(df)):
        try:
            sub_df_lat = sub_df_lat.append(pd.DataFrame(df.iloc[i,idx]).iloc[1,:])
        except:
            sub_df_lat = sub_df_lat.append(pd.DataFrame(ast.literal_eval(df.iloc[i,idx])).iloc[1,:])
        
    sub_df_lat['lat'] = sub_df_lat['coordinates']
    sub_df_lat.drop(['coordinates','type'],axis = 1,inplace = True)
    sub_df_lat.reset_index(drop=True, inplace=True)

    sub_df = pd.concat([sub_df_long,sub_df_lat],axis = 1)
    
    return sub_df

In [9]:
#parses the 'place' heading into its own dataframe
def parse_place_data(df):
    
    idx = df.columns.get_loc("place")
    
    thisdict = str({'id': 'NA', 'url': 'NA', 'place_type': 'NA', 'name': 'NA', 'full_name': 'NA', 'country_code': 'NA', 
                'country': 'NA', 'contained_within': [], 
                'bounding_box': {'type': 'NA', 'coordinates': [[['NA', 'NA'], ['NA', 'NA'], ['NA', 'NA'], ['NA', 'NA']]]}, 
                'attributes': {}})
    df['place'].replace([None], thisdict, inplace=True)
    
    sub_df = pd.DataFrame()

    id_list = []
    url_list = []
    place_type_list = []
    name_list = []
    cc_list = []
    country_list = []

    for i in range(0,len(df)):
        
        if type(df.iloc[i,idx]) == str:
            sub_dict = ast.literal_eval(df.iloc[i,idx])
        else:
            sub_dict = df.iloc[i,idx]

        id_list.append(sub_dict['id'])
        url_list.append(sub_dict['url'])
        place_type_list.append(sub_dict['place_type'])
        name_list.append(sub_dict['name'])
        cc_list.append(sub_dict['country_code'])
        country_list.append(sub_dict['country'])

    sub_df['place_id'] = id_list
    sub_df['place_url'] = url_list
    sub_df['place_type'] = place_type_list
    sub_df['place_full_name'] = name_list
    sub_df['place_country_code'] = cc_list
    sub_df['place_country'] = country_list
    
    sub_df.reset_index(drop=True, inplace=True)

    return sub_df

#### Data Cleaning

In [10]:
#drops unecessary columns from primary dataframe 
def drop_columns(df):
    drop_list = ['str','in_reply','User','geo','Coordinates','Place','entities',
                 'utc','time_zone','enabled','contributors','lang','quoted_status']

    for st in df.columns:
        try:
            if any(ext in st for ext in drop_list):
                df.drop(st, axis = 1, inplace = True)
        except:
            pass
    
    return df

In [11]:
#clean the text column and insert new cleaned data
def clean_text(df):
    
    twitter_list = []
    stop_words = set(stopwords.words('english'))

    for row in df['full_text']:

        twitter_text = re.sub('(@[\w]+)|(^rt\s+)|(http[s]:\/\/[\w\.-\/]+[\s])|(http:\/\/[\w\.-\/]+[\s])|(#)|(â)','',row) #Remove html attributes 
        twitter_text = re.sub(r"http\S+", "", twitter_text) #Remove URL
        twitter_text = re.sub('([^\w]+)',' ',twitter_text) #Remove punctuation
        twitter_text = re.sub('(\s)x\w+|xe2',' ',twitter_text) #Remove emojis
        twitter_text = twitter_text.lower() #Lowercase all characters
        twitter_tokenizer = RegexpTokenizer(r'\w+') #Setup tokenizer          
        twitter_token = twitter_tokenizer.tokenize(twitter_text) #Tokenize words in string
        twitter_text = [word for word in twitter_token if word not in stop_words] #Remove stop words

        ps = PorterStemmer()
        twitter_text = [ps.stem(word) for word in twitter_text] #Stem 
        twitter_list.append(' '.join(twitter_text))

    df.insert(loc = 3, column = 'clean_text', value = pd.DataFrame(twitter_list))
    
    return df

In [12]:
def clean_col_text(url_text,df):
    
    twitter_list = []
    stop_words = nltk.corpus.stopwords.words('english')

    for row in url_text:
        twitter_text = re.sub('(@[\w]+)|(^rt\s+)|(http[s]:\/\/[\w\.-\/]+[\s])|(http:\/\/[\w\.-\/]+[\s])|(#)|(â)','',row) #Remove html attributes 
        twitter_text = re.sub(r"http\S+", "", twitter_text) #Remove URL
        twitter_text = re.sub('([^\w]+)',' ',twitter_text) #Remove punctuation
        twitter_text = re.sub('(\s)x\w+|xe2',' ',twitter_text) #Remove emojis
        twitter_text = twitter_text.lower() #Lowercase all characters
        twitter_tokenizer = RegexpTokenizer(r'\w+') #Setup tokenizer          
        twitter_token = twitter_tokenizer.tokenize(twitter_text) #Tokenize words in string
        twitter_text = [word for word in twitter_token if word not in stop_words] #Remove stop words

        ps = PorterStemmer()
        twitter_text = [ps.stem(word) for word in twitter_text] #Stem 
        twitter_list.append(' '.join(twitter_text).replace('titl',''))

    df.insert(loc = len(df.columns), column = url_text.name + '_cleaned', value = twitter_list)

    return df

In [13]:
def anon_col (df, cols):
    for col_name in cols:
        keys = {cats: i for i, cats in enumerate(df[col_name].unique())}
        df[col_name] = df[col_name].apply(lambda x: keys[x])
        
    return df

#### Convert JSON to Dataframe (Main)

In [14]:
#convert JSON file to readable dataframe
def json_2_df(file_loc):
    
    #read only json file and convert to dataframe
    with open(file_loc, 'r') as myfile:
        data = myfile.read()
        df = pd.read_json(data, lines = True)
        
        #parsing data subsets into individual dataframes
        sub_df_user = parse_user_data(df)
        sub_df_coord = parse_coord_data(df)
        sub_df_place = parse_place_data(df)
        
        #combining all dataframes and renaming columns
        df_combined = pd.concat([df, sub_df_user, sub_df_coord, sub_df_place], axis=1)
        df_combined.rename(columns={"user": "User", "place": "Place", "coordinates": "Coordinates"}, inplace = True)
        
        #data cleaning (dropping unecessary columns and cleaning twitter text)
        drop_columns(df_combined)
        clean_text(df_combined)
        
        #additional cleanups
        df_combined["target_label"] = "" #insert blank target label column
        df_combined['id'] = df_combined['id'].astype(str) #convert id field to string
        
        #modify datetime to readable Excel format
        df_combined.insert(loc = 1, column = 'time_created_at', 
                           value = pd.to_datetime(df_combined['created_at']).dt.time) #insert created at time column
        df_combined['created_at'] = pd.to_datetime(df_combined['created_at']).dt.date #convert datetime to just date
    
    return df_combined

## 5. Exporting Output Files

This program utilizes the functions defined above and automatically creates a specific number of dataframes based on the count of JSON files in the 'downloaded_json_folders' path that has been set up in Google Drive.  Once each dataframe is created, it is then exported to Excel with a unique name and added to the 'converted_to_excel' repository.  This ensures that the user will not have to manually create each dataframe or Excel template (ie. if there are 10 JSON files, the program will automatically create 10 dataframes and Excel output files).

These Excel files are then merged into a master file, which is exported to the same folder path.  The program will check to see if the master file exists in location.  If it does, this section is bypassed as it can take 20+ minutes for this entire program section to run.  Please note that if you are using this code, you will need to change the folder paths above.  It is highly recommended that you maintain the naming convention of the files so that you will not need to make any additional changes. 

**Please Note: This program only utilizes 'coords' and 'place' data specified in their naming convention.  The reason for this is to ignore other potential, irrelevant Excel files that may be stored with this data.**

In [20]:
#This cell first checks to see if the master consolidated file exists
if os.path.exists(converted_path + '\\master_twitter_cleaned_data.xlsx') != True:

    count_place = 0 #place counter set to 0
    count_coords = 0 #coords counter set to 0

    df_place = {} #define empty place dictionary
    df_coords = {} #define empty coords dictionary

    #will loop through all JSON files in folder path
    for f in os.listdir(json_path):

        #if file contains 'place' in name, convert JSON to df with 'place' naming convention
        if 'place' in str(f):
            df_place[count_place] = json_2_df(json_path + '\\all_tweets_place_' + str(count_place) + '.jsonl')
            df_place[count_place].set_index('id',inplace = True)
            df_place[count_place].to_excel(converted_path + '\\twitter_place_clean_data_' + str(count_place) + '.xlsx',encoding='utf-8-sig')

            count_place = count_place + 1
        
        #if file contains 'coords' in name, convert JSON to df with 'coords' naming convention
        elif 'coords' in str(f):
            df_coords[count_coords] = json_2_df(json_path + '\\all_tweets_coords_' + str(count_coords) + '.jsonl')
            df_coords[count_coords].set_index('id',inplace = True)
            df_coords[count_coords].to_excel(converted_path + '\\twitter_coords_clean_data_' + str(count_coords) + '.xlsx',encoding='utf-8-sig')

            count_coords = count_coords + 1

In [30]:
#This cell first checks to see if the master consolidated file exists
#If master consolidated file does not exist, this cell will loop through all dataframes and merge them into the master file

if os.path.exists(converted_path + '\\master_twitter_cleaned_data.xlsx') != True:
    
    df = {}
    count_place = 0
    count_coords = 0

    dataframe = list()

    for f in os.listdir(converted_path):

        if 'place' in str(f) and 'desktop.ini' not in str(f) and '~$' not in str(f):
            df[count_place] = pd.read_excel(converted_path + '\\twitter_place_clean_data_' + str(count_place) + '.xlsx')
            dataframe.append(df[count_place])
            count_place += 1

        if 'coords' in str(f) and 'desktop.ini' not in str(f) and '~$' not in str(f):
            df[count_coords] = pd.read_excel(converted_path + '\\twitter_coords_clean_data_' + str(count_coords) + '.xlsx')
            dataframe.append(df[count_coords])
            count_coords += 1

    excel_df = pd.concat(dataframe)
    excel_df.to_excel(converted_path + '\\master_twitter_cleaned_data.xlsx',encoding='utf-8-sig')

else:
    excel_df = pd.read_excel(converted_path + '\\master_twitter_cleaned_data.xlsx')

In [ ]:
excel_df.drop('Unnamed: 0', axis=1, inplace=True)

In [34]:
#Master consolidated Excel file attributes
print(f"Rows: {excel_df.shape[0]}, Columns: {excel_df.shape[1]}")
excel_df.reset_index(drop=True, inplace=True)
excel_df.head(3)

Rows: 122940, Columns: 38


,id,created_at,time_created_at,full_text,clean_text,truncated,display_text_range,source,is_quote_status,retweet_count,...,user_statuses_count,long,lat,place_id,place_url,place_type,place_full_name,place_country_code,place_country,target_label
0,1115990344401702914,2019-04-10,14:50:06,Fighting Stigma : Harris to explore plans to b...,fight stigma harri explor plan ban children va...,False,"[0, 142]","<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",False,0,...,800115,-73.479357,45.667751,62b78f4b441b9a2f,https://api.twitter.com/1.1/geo/id/62b78f4b441...,city,Varennes,CA,Canada,NaN
1,1226314915452837888,2020-02-09,01:20:34,Participated in the “V for Vaccine” flash mob ...,particip v vaccin flash mob today brooklyn ny ...,False,"[0, 187]","<a href=""http://instagram.com"" rel=""nofollow"">...",False,9,...,21672,-73.975865,40.683033,011add077f4d2da3,https://api.twitter.com/1.1/geo/id/011add077f4...,city,Brooklyn,US,United States,NaN
2,1097900340723310593,2019-02-19,16:46:53,Ty helps combat the spread of the flu by follo...,ty help combat spread flu follow rule like ty ...,False,"[0, 210]","<a href=""http://instagram.com"" rel=""nofollow"">...",False,0,...,4699,-81.488455,36.851915,3437938e5550bb2b,https://api.twitter.com/1.1/geo/id/3437938e555...,city,Marion,US,United States,NaN


## 6. Webscraping URLs & Re-Appending to Master Dataframe

The section below focuses on extracting any URL's posted in the 'full_text' column and reappending that information to the master dataframe.  This is necessary since approximately 22% of Tweets contain a URL pointing to another specific page with vaccine hesitency context.  

There are 2 types of URL's to be extracted: direct Tweets and others.  Due to Twitter's privacy policy on protecting Tweets, if the URL is a direct Twitter link, the ID will be extracted and run through the Twython API.  Any other URL's will simply extract the title HTML tag.  In both instances, dataframes will be constructed and reappended to the master dataframe.

The process of extracting information from the URL's can take upwards of 7+ hours, and so to avoid this runtime, the program will check to see if the "Tweet" and "Title" dataframes have been exported to Excel.  If so, it will bypass the extraction code.

**Please Note: You must have your own Twitter consumer information in order to run the Twython API.  To protect our own user content, this information has been hidden in a password protected file stored over the cloud.  If you don't have this information, simply sign up with your own Twitter account and request for Twython access.  Once you have this, replace the consumer_info objects with your own consumer keys (there will be prompts telling you where to replace the code below).**

#### Extracting URL Lists

In [35]:
#First, extract the unique URL's from the post 
url_list_full = list()

for my_string in excel_df['full_text']:
    try:
        url_link = re.findall(r'(https?://[^\s]+)', my_string)[0]
    except:
        url_link = ''

    url_list_full.append(url_link)

In [ ]:
#Once the unique URL's have been extracted, convert them to their proper domain URL
#This will list every language variation of the same URL

if os.path.exists(converted_url_path + '\\twitter_place_url_extracted_tweets.xlsx') != True and os.path.exists(converted_url_path + '\\twitter_place_url_extracted_titles.xlsx') != True:

    converted_url_list_full = list()

    for url in url_list_full:

        try:
            html = requests.get(url)
            soup = BeautifulSoup(html.content,"html.parser")
            new_url = soup.find('link', hreflang = 'en')
            #print(new_url)
            converted_url_list_full.append(new_url)
        except:
            converted_url_list_full.append('')

In [24]:
#Extract the first listed URL (in english)

if os.path.exists(converted_url_path + '\\twitter_place_url_extracted_tweets.xlsx') != True and os.path.exists(converted_url_path + '\\twitter_place_url_extracted_titles.xlsx') != True:

    converted_eng_list_full = list()

    for i in range(0,len(converted_url_list_full)):
        try:
            converted_eng_list_full.append(re.findall(r'(https?://[^\s]+)', str(converted_url_list_full[i]))[0])
        except:
            converted_eng_list_full.append('')

#### Parsing Title and Tweet Data from URLs

In [25]:
if os.path.exists(converted_url_path + '\\twitter_place_url_extracted_tweets.xlsx') != True and os.path.exists(converted_url_path + '\\twitter_place_url_extracted_titles.xlsx') != True:

    #For title extraction
    title_list = list()
    title_url_converted_list = list()
    title_unique_url = list()

    #For Twitter ID extraction
    id_list = list()
    id_url_converted_list = list()
    id_unique_url = list()

    count = 0

    for eng_url in converted_eng_list_full:

        if 'twitter' not in str(eng_url):
            try:
                html = requests.get(eng_url)
                soup = BeautifulSoup(html.content,"html.parser")
                new_url = soup.find('title')

                title_list.append(new_url)
                title_url_converted_list.append(eng_url)
                title_unique_url.append(url_list_full[count])

                count += 1
            except:
                title_list.append('')
                title_url_converted_list.append(eng_url)
                title_unique_url.append(url_list_full[count])

                count += 1
        else:
            if any(ext in re.findall(r'(https?://[^\s]+)',eng_url)[0] for ext in ['video','photo']):
                temp = eng_url.split('status/')[-1]
                digit = temp.split('/')[-3]
            else:  
                temp = eng_url.split('status/')[-1]
                digit = temp.split('?')[-2]

            id_list.append(digit)
            id_url_converted_list.append(eng_url)
            id_unique_url.append(url_list_full[count])

#### Title Extraction (url2title)

In [34]:
'''url2title'''
if os.path.exists(converted_url_path + '\\twitter_place_url_extracted_tweets.xlsx') != True and os.path.exists(converted_url_path + '\\twitter_place_url_extracted_titles.xlsx') != True:

    df_title = pd.DataFrame()

    df_title['unique_url'] = title_unique_url
    df_title['full_url'] = title_url_converted_list
    df_title['extracted_titles'] = title_list
    
    df_title = clean_col_text(df_title['extracted_titles'].astype(str),df_title)
    df_title.to_excel(converted_url_path + '\\twitter_place_url_extracted_titles.xlsx',encoding='utf-8-sig')
    
else:
    df_title = pd.read_excel(converted_url_path + '\\twitter_place_url_extracted_titles.xlsx')
    df_title.drop('Unnamed: 0', axis = 1, inplace = True)
    
df_title.head(3)

,unique_url,full_url,extracted_titles,extracted_titles_cleaned
0,https://t.co/1a916WBsA6,NaN,NaN,NaN
1,https://t.co/L0UCON7g78,NaN,NaN,NaN
2,https://t.co/oYlzZrdwE0,NaN,NaN,NaN


#### Twitter Extraction (url2tweetID > id2tweet)

In [35]:
#if tweets and title databases DO NOT EXIST in directory then create
if os.path.exists(converted_url_path + '\\twitter_place_url_extracted_tweets.xlsx') != True and os.path.exists(converted_url_path + '\\twitter_place_url_extracted_titles.xlsx') != True:
    
    '''url2tweetID'''
    extracted_tweets = {'unique_url':id_unique_url,'full_url':id_url_converted_list,'twitter_id':id_list}

    df_extracted_tweets = pd.DataFrame(extracted_tweets)
    df_extracted_tweets.rename(columns = {"twitter_id": "id"}, inplace = True)
    df_extracted_tweets.id = df_extracted_tweets.id.astype(int64)
    
    df_id_list = df_extracted_tweets.drop(columns = ['unique_url', 'full_url'])
    df_id_list.set_index("id",inplace = True)
    df_id_list.to_csv('extracted_tweet_ids.csv',encoding='utf-8-sig')
    
    '''ID2tweet'''
    '''Please put in your own credentials where XXXXXXX is indicated below'''
    !python download_tweets_vaxxmisinfo.py -i extracted_tweet_ids.csv -o extracted_tweet_ids_downloaded.jsonl --consumerkey XXXXXXX --consumersecret XXXXXXX --accesstoken XXXXXXX  --accesssecret XXXXXXX
    
    #read from JSONL in local directory
    file_temp = 'extracted_tweet_ids_downloaded.jsonl'
    with open(file_temp, 'r') as myfile:
            data = myfile.read()
            df_tweets_temp = pd.read_json(data, lines = True)
    
    df_tweets_temp = df_tweets_temp[['id','full_text']]
    df_tweets_temp.rename(columns = {"full_text": "extracted_tweets"}, inplace = True)
    
    df_tweets = pd.merge(df_tweets_temp,df_extracted_tweets,on ='id',how = 'outer')
    clean_col_text(df_tweets.extracted_tweets,df_tweets)
    
    df_tweets.to_excel(converted_url_path + '\\twitter_place_url_extracted_tweets.xlsx',encoding='utf-8-sig')
    
    #move data from local to GDrive
    if os.path.exists(converted_url_path + '\\extracted_tweet_ids_downloaded.jsonl') != True and os.path.exists(converted_url_path + '\\extracted_tweet_ids.csv') != True:
        
        os.rename('extracted_tweet_ids_downloaded.jsonl', converted_url_path + '\\extracted_tweet_ids_downloaded.jsonl' )
        os.rename('extracted_tweet_ids.csv', converted_url_path + '\\extracted_tweet_ids.csv' )

#if tweets and title databases do exist in directory then read from path
else:
    df_tweets = pd.read_excel(converted_url_path + '\\twitter_place_url_extracted_tweets.xlsx')
    df_tweets.drop('Unnamed: 0', axis = 1, inplace = True)

df_tweets.head(3)

,id,extracted_tweets,unique_url,full_url,extracted_tweets_cleaned
0,1182230763162477056,#BantryGeneralHospital #YourBestShot Our Quali...,https://t.co/kuVrCHdX5t,https://twitter.com/carolcroke1/status/1182230...,bantrygeneralhospit yourbestshot qualiti patie...


#### Merging Data Frames (url2text)

In [59]:
'''url2text'''
#add unique url to main dataframe and convert type to string
excel_df['unique_url'] = url_list_full
excel_df['unique_url'] = excel_df['unique_url'].astype(str)

#drop all columns from title df except for unique url and extracted clean titles
df_title = df_title[['unique_url','extracted_titles_cleaned']]
df_title['unique_url'] = df_title['unique_url'].astype(str)

#drop all columns from tweets df except for unique url and extracted clean tweets
df_tweets = df_tweets[['unique_url','extracted_tweets_cleaned']]
df_tweets['unique_url'] = df_tweets['unique_url'].astype(str)

In [60]:
#create new empty df 
df = pd.DataFrame()

#merge all df's on unique url and drop duplicates
df = pd.merge(excel_df,df_title,on = 'unique_url', how = 'left')
df_final = df.merge(df_tweets, on = 'unique_url', how = 'left').drop_duplicates()

In [61]:
#reset index and fill NA's with blanks
df_final.reset_index(drop = True, inplace = True)
df_final['id'] = df_final['id'].astype(str)
df_final.set_index('id', inplace=True)
df_final = df_final.fillna('')

In [62]:
keyword_list = list()

for tweet in df_final['full_text']:
    
    if any(ele in tweet for ele in neg_keywords) and any(ele in tweet for ele in pos_keywords):
        keyword_list.append("negative")
    elif any(ele in tweet for ele in neg_keywords):
        keyword_list.append("negative")
    elif any(ele in tweet for ele in pos_keywords):
        keyword_list.append("positive")
    else:
        keyword_list.append("none")

df_final['keywords'] = keyword_list

In [63]:
#export master annotation file to directory
df_final['X'] = df_final['clean_text'] + ' ' + df_final['extracted_titles_cleaned'] + ' ' + df_final['extracted_tweets_cleaned'] + ' ' + df_final['user_screen_name']  
df_final = anon_col(df_final,['user_name', 'user_screen_name']) #anonamize Twitter usernames

In [65]:
df_final.to_excel(converted_url_path + '\\master_twitter_database.xlsx',encoding='utf-8-sig')
df_final.head(3)

,created_at,time_created_at,full_text,clean_text,truncated,display_text_range,source,is_quote_status,retweet_count,favorite_count,...,place_type,place_full_name,place_country_code,place_country,target_label,unique_url,extracted_titles_cleaned,extracted_tweets_cleaned,keywords,X
id,,,,,,,,,,,,,,,,,,,,,
1178908412794434048,2019-10-01,05:44:03,Stigmabase — Hope for HIV Vaccine Being Teste...,stigmabas hope hiv vaccin test south africa hi...,False,"[0, 241]","<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",False,0,0,...,city,Johannesburg,ZA,South Africa,,https://t.co/1a916WBsA6,,,none,stigmabas hope hiv vaccin test south africa hi...
1178922253167149056,2019-10-01,06:39:03,Stigmabase — New research reveals over half o...,stigmabas new research reveal half peopl irela...,False,"[0, 278]","<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",False,1,0,...,city,Dublin City,IE,Ireland,,https://t.co/L0UCON7g78,,,positive,stigmabas new research reveal half peopl irela...
1178951214857196032,2019-10-01,08:34:08,Thoughts? San Francisco:: Tennessee raccoons t...,thought san francisco tennesse raccoon get vac...,False,"[0, 97]","<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",False,0,0,...,city,Portland,US,United States,,https://t.co/oYlzZrdwE0,,,none,thought san francisco tennesse raccoon get vac...
